# Glavna koda
V tem delu je zapisan prvi del kode, kjer bomo testirali grafe za določene velikosti.
Koda je ravno tako opisana v [poročilu](https://github.com/mpracek/Graffiti-conjecture-194/blob/master/Porocilo/dolgo_porocilo.pdf).

In [1]:
import random
import operator
import math

def nasi_grafi(stevilo_vozlisc):
    """
    Funkcija, ki nam vrne enostavne povezane grafe na dolocenem stevilu vozlisc.
    """
    grafi = list(graphs.nauty_geng(str(stevilo_vozlisc)+" -c"))
    return grafi

def neodvisnostno_stevilo(G):
    """
    Vrne neodvistno število grafa G
    Za pomoc uporabimo independet_set() iz modula neusmerjenih grafov.
    """
    neodvisno = G.independent_set()
    dolzina = len(neodvisno)
    return dolzina


def naredi_podgraf(G, seznam):
    """
    Funkcija, ki nam za dan seznam vozlisc vrne podgraf, definiran na le teh
    Seznam je tukaj nabor vozlisc, ki nas zanimajo.
    """
    H = Graph(G, immutable=True)
    sosedi = []
    for vozlisce in H:
        nabor_sosedov = ()
        if vozlisce in seznam:
            for element in H[vozlisce]:
                if element in seznam:
                    a = list(nabor_sosedov)
                    a.append([vozlisce, element])
                    tuple(a)
        sosedi.append(nabor_sosedov)
    nov_graf = H.subgraph(vertices = seznam, edges = sosedi)
    return nov_graf


def lokalna_neodvisnost(G, vozlisce):
    """
    Vrne lokalno neodvistnost grafa G v vozliscu v.
    Lokalna neodvistnost je neodvistnost podgrafa, ki ga določajo sosedi vozlišča v,
    za nek v iz množice vozlišč.
    """
    mnozica = G[vozlisce]
    novGraf = naredi_podgraf(G, mnozica)
    lokalna = neodvisnostno_stevilo(novGraf)
    return lokalna

def povprecna(G):
    """
    Vrne povprečno lokalno neodvisnost grafa G
    """
    povprecje =  0
    for vozlisce in G:
        povprecje += lokalna_neodvisnost(G, vozlisce)
    povprecna_vrednost  = povprecje/ len(G)
    return povprecna_vrednost

def preverjanje_za_en_graf(G):
    """
    Preveri, ali za en graf velja, če je ta graf izjema.
    """
    if neodvisnostno_stevilo(G) <= 1 + povprecna(G):
            if G.hamiltonian_path() == None:
                graf = str(G)
                return "Ovrgli smo domnevo in izjema je" + 	graf
    else:
        return "Izjeme nismo ovrgli"

def preverjanje(stevilo_vozlisc):
    """
    Preveri veljavnost konjekture za vse grafe na določenem številu vozlišč.
    """
    for G in nasi_grafi(stevilo_vozlisc):
        if neodvisnostno_stevilo(G) <= 1 + povprecna(G):
            if G.hamiltonian_path() == None:
                p = G.plot()
                p.show()
                return "Ovrgli smo domnevo"
    return "Izjeme nismo ovrgli"


In [2]:
print(preverjanje(4))

Izjeme nismo ovrgli


# Genetski algoritem
Od tu naprej bodo vse funkcije bile uporabljene za definicijo [genetskega algoritma](https://en.wikipedia.org/wiki/Genetic_algorithm).

In [3]:
def poisson(t = 1, lambd = 1/2):
    """
    S to funkcijo določimo začetno stevilo vozlisc
    """
    stevilo_vozlisc = 0
    racunalo = 0
    while racunalo < t:
        stevilo_vozlisc += 1
        racunalo += random.expovariate(lambd)
    return stevilo_vozlisc


In [4]:
def zacetna_populacija(maksimalna):
    """
    Ta funkcija nam da grafe, na katerih bo opravljen prvi test.
    Maksimalna nam pove, koliko je največja (maksimalna) velikost vsake generacije.
    """
    stevilo_vozlisc = poisson(t = 5, lambd = 1/2)
    populacija = []
    stevec = 0
    while stevec < maksimalna:
        graf = graphs.RandomGNP(stevilo_vozlisc, random.uniform(0, 1))
        if graf.is_connected():
            populacija.append(graf)
            stevec += 1
    return populacija


In [5]:
def razporedi(maksimalna=None):
    """
    Izracuna kriterij, po katerem vse elemente razporedimo in razporedi elemente populacije.
    Navadno za populacijo uporabimo zacetna_populacijo(maksimalna).
    """
    slovar = dict()
    populacija = zacetna_populacija(maksimalna)
    for element in populacija:
        slovar[element] = neodvistnostno_stevilo(element)
    razporejena_populacija = sorted(slovar.items(), key = operator.itemgetter(1))
    nasa_populacija = []
    for osebek in razporejena_populacija:
        nasa_populacija.append(populacija[osebek[0]])
    return nasa_populacija


In [6]:
def testna_populacija(maksimalna):
    """
    Za zacetni parameter izberemo zacetna_populacija(), ki nam bo vrnila
    vse grafe zacetne velikosti.
    Najprej moramo določiti, kolikšen del začetne populacije bomo testirali
    V genetiki velja, da ostanejo zgolj najboljsi, zato bomo vzeli le najboljse,
    torej tiste, z minimalnim neodvistnostnim številom.
    """
    populacija = razporedi()
    if len(populacija) > maksimalna:
        testna_populacija = populacija[:maksimalna]
    else:
        testna_populacija = populacija
    return testna_populacija


In [7]:
def doda_povezavo(graf):
    """
    Spremeni graf tako, da mu doda povezavo
    """
    G = Graph(graf)
    if G.vertices() == []:
        return G
    else:
        vozlisca = graf.vertices()
        vozlisce1 = G.random_vertex()
        vozlisce2 = G.random_vertex()
        if vozlisce1 != vozlisce2:
            G.add_edge(vozlisce1, vozlisce2)
        return G


In [8]:
def odstrani_povezavo(graf):
    """
    Odstrani nakljucno povezavo iz grafa;
    Pazimo, da mora graf, ki ga dobimo, biti povezan.
    """
    kopija = Graph(graf, immutable=True)
    G = Graph(kopija)
    if G.vertices() == []:
        return G
    else:
        vozlisce1 = G.random_vertex()
        vozlisce2 = G.random_vertex()
        if vozlisce1 != vozlisce2:
            G.delete_edge(vozlisce1, vozlisce2)
            if G.is_connected():
                return G
            else:
                G.add_edge(vozlisce1, vozlisce2)


In [9]:
def mutacija_vozlisce(graf):
    """
    Doda vozlisce in mu nato doda nekaj povezav nanj.
    """
    graf = Graph(graf)
    dolzina = len(graf.vertices())
    graf.add_vertex()
    stevilo = randint(0,dolzina)
    vozlisca = graf.vertices()
    dodaj_vozlisca = random.sample(vozlisca, stevilo)
    for i in range(stevilo):
        graf.add_edge(novo, dodaj_vozlisca[i])
    return graf


In [10]:
def odstrani_vozlisce(graf):
    """
    Funkcija iz grafa odstrani vozlišče in vse povezave nanj.
    """
    graf = Graph(graf)
    if graf.vertices() == []:
        return graf
    else:
        vozlisce1 = graf.random_vertex()
        graf.delete_vertex(vozlisce1)
        if graf.is_connected():
            return graf


In [11]:
def mutacija(graf):
    """
    Na grafu lahko odstranimo, dodamo povezavo, dodamo, odstranimo vozlisce in vsako kombinacijo le teh.
    """
    p = random.uniform(0,1)
    if p <= 1/15:
        odstrani_vozlisce(graf)
    elif p > 1/15 and p <= 2/15:
         mutacija_vozlisce(graf)
    elif p > 2/15 and p <= 3/15:
        odstrani_povezavo(graf)
    elif p > 3/15 and p <= 4/15:
        doda_povezavo(graf)
    elif p > 4/15 and p <= 5/15:
        sprememba = odstrani_vozlisce(graf)
        mutacija_vozlisce(sprememba)
    elif p > 5/15 and p <= 6/15:
        sprememba = odstrani_vozlisce(graf)
        odstrani_povezavo(sprememba)
    elif p > 6/15 and p <= 7/15:
        sprememba = odstrani_vozlisce(graf)
        doda_povezavo(sprememba)
    elif p > 7/15 and p <= 8/15:
        sprememba = mutacija_vozlisce(graf)
        odstrani_povezavo(sprememba)
    elif p > 8/15 and p <= 9/15:
        sprememba = mutacija_vozlisce(graf)
        doda_povezavo(sprememba)
    elif p > 9/15 and p <= 10/15:
        sprememba = odstrani_povezavo(graf)
        doda_povezavo(sprememba)
    elif p > 10/15 and p <= 11/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 11/15 and p <= 12/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 12/15 and p <= 13/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = odstrani_vozlisce(sprememba)
        odstrani_povezavo(sprememba2)
    elif p > 13/15 and p <= 14/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        odstrani_vozlisce(sprememba2)
    elif p > 14/15 and p <= 15/15:
        sprememba = mutacija_vozlisce(graf)
        sprememba2 = doda_povezavo(sprememba)
        sprememba3 = odstrani_povezavo(sprememba2)
        odstrani_vozlisce(sprememba3)


In [12]:
def nova_populacija(populacija = None, verjetnost = 0.05):
    """
    Vsak graf iz populacije z neko verjetnostjo spremenimo.
    Za zacetku za populacijo uporabimo testna_populacija(maksimalna)
    Ti spremenjeni grafi nam dajo novo populacijo.
    """
    naslednja_generacija = []
    for i in range(len(populacija)):
        q = random.uniform(0,1)
        r = random.uniform(0,1)
        prob = q * r
        if verjetnost < prob:
            naslednja_generacija[i] = mutacija(populacija[i])
        else:
            naslednja_generacija[i] = populacija[i]
    return naslednja_generacija


In [13]:
def crossover(n, stars1, stars2):
    """
    V nasem primeru je n stevilo vozlisc, na katerih se grafa krizata.
    Nove povezave nam povedo, koliko in katere povezav bo imel graf.
    """
    while True:
        graf1 = stars1.random_subgraph(0.5)
        graf2 = stars2.random_subgraph(0.5)
        if len(graf1.vertices()) + len(graf2.vertices()) == n and len(graf1.vertices()) >= 1 and len(graf1.vertices()) < n and graf1.is_connected() and graf2.is_connected():
            graf1.relabel()
            graf2.relabel()
            naslednik = graf1.disjoint_union(graf2)
            nove_povezave = poisson(lambd = log(n/2))
            for k in range(nove_povezave):
                prvi = graf1.random_vertex()
                drugi = graf2.random_vertex()
                potomec.add_edge((0, prvi), (1, drugi))
            naslednik.relabel()
            break
    return potomec


In [14]:
def potomci(populacija = None, verjetnost = 0.05):
    """
    Navadno za populacijo uporabimo nova_populacija(populacija = testna_populacija(maksimalna))
    Funkcija nam vrne prejšnjo populacijo in premešano populacijo, združeno v eno.
    """
    nova_populacija = populacija
    stevilo_parjenj = poisson(t = len(populacija))
    for k in range(stevilo_parjenj):
        starsa = random.sample(populacija, k = 2)
        nova_populacija.append(crossover(n, starsa[0], starsa[1]))
    return nova_populacija


In [15]:
def KoncniTestGA(konec,maksimalna):
    """
    Konec pove, koliko generacij naj pogledamo.
    Funkcija lahko vrne dve možnosti:
                                     1. Najde nam izjemo, jo pokaže in tako ovržemo domnevo.
                                     2. Izjeme sploh ne dobimo, in tako izjeme ne vrnemo.
    """
    populacija = testna_populacija(maksimalna)
    stevec = 1
    while stevec <= konec:
        populacija = potomci(populacija)
        populacija = mutacija(populacija)
        populacija = testna_populacija(populacija)
        for i in range(len(populacija)):
            G = populacija[i]
            if neodvistnostno_stevilo(G) <= 1 + povprecna(G):
                if G.hamiltonian_path() == None:
                    p = G.plot()
                    p.show()
                    return "Ovrgli smo domnevo"

        stevec += 1
    return "Domneva ni ovrzena."

In [16]:
print(KoncniTestGA(1000000,1000000))

Domneva ni ovrzena.


Zanima nas še, koliko časa porabi genetski algoritem za svoje izvajanje. Da to testiramo, bomo uporabili novo funkcijo.

In [17]:
import time
def casovna_zahtevnost_konec(konec,maksimalna):
    """
    Konec nam tu pove, za koliko primerov testiramo našo kodo; torej koliko podprimerov testiramo.
    Maksimalna nam pove koliko naj bo velikost največje generacije.
    """
    slovar = dict()
    for i in range(konec):
        zacetek = time.time()
        KoncniTestGA(i, maksimalna)
        konec = time.time()
        cas = konec - zacetek
        slovar[i] = cas
    return slovar

def meritev(konec,tekac):
    zacetek = time.time()
    KoncniTestGA(konec, tekac)
    konec = time.time()
    cas = konec - zacetek
    return cas

def casovna_zahtevnost_maksimalna(konec, maksimalna):
    """
    Ravno obratna koda kot prej, spreminja se maksimalna velikost generacije.
    """
    slovar = dict()
    for tekac in range(maksimalna):
        slovar[tekac] = meritev(konec,tekac)
    return slovar


In [18]:
print(casovna_zahtevnost_konec(100,100))

{0: 0.000102996826171875, 1: 0.00037598609924316406, 2: 0.0005071163177490234, 3: 0.0007390975952148438, 4: 0.0009348392486572266, 5: 0.0012040138244628906, 6: 0.0016129016876220703, 7: 0.0014870166778564453, 8: 0.0018050670623779297, 9: 0.002017974853515625, 10: 0.002276897430419922, 11: 0.0037641525268554688, 12: 0.003376007080078125, 13: 0.002891063690185547, 14: 0.0022258758544921875, 15: 0.002004861831665039, 16: 0.002788066864013672, 17: 0.0024271011352539062, 18: 0.00292205810546875, 19: 0.0030579566955566406, 20: 0.0032711029052734375, 21: 0.0031180381774902344, 22: 0.003033161163330078, 23: 0.003963947296142578, 24: 0.003740072250366211, 25: 0.0047550201416015625, 26: 0.0052869319915771484, 27: 0.005316972732543945, 28: 0.004416942596435547, 29: 0.0065801143646240234, 30: 0.0071868896484375, 31: 0.0048258304595947266, 32: 0.004372119903564453, 33: 0.004862070083618164, 34: 0.013685941696166992, 35: 0.0071179866790771484, 36: 0.0052340030670166016, 37: 0.0058820247650146484, 38

In [19]:
print(casovna_zahtevnost_maksimalna(100,100))

{0: 0.03697395324707031, 1: 0.018002033233642578, 2: 0.016150951385498047, 3: 0.015027046203613281, 4: 0.016041994094848633, 5: 0.04330897331237793, 6: 0.017927169799804688, 7: 0.01784205436706543, 8: 0.01814103126525879, 9: 0.036965131759643555, 10: 0.027753114700317383, 11: 0.016657114028930664, 12: 0.02097010612487793, 13: 0.019810914993286133, 14: 0.017683029174804688, 15: 0.014040946960449219, 16: 0.013833045959472656, 17: 0.01547098159790039, 18: 0.01568007469177246, 19: 0.01655101776123047, 20: 0.015461921691894531, 21: 0.013890981674194336, 22: 0.01726984977722168, 23: 0.012929916381835938, 24: 0.014117956161499023, 25: 0.02802300453186035, 26: 0.018117904663085938, 27: 0.015720129013061523, 28: 0.01556396484375, 29: 0.01759791374206543, 30: 0.016549110412597656, 31: 0.01403498649597168, 32: 0.016992807388305664, 33: 0.014622926712036133, 34: 0.01503896713256836, 35: 0.016518831253051758, 36: 0.016237974166870117, 37: 0.0166780948638916, 38: 0.025257110595703125, 39: 0.01501989

In [20]:
print(casovna_zahtevnost_maksimalna(100,1000))

{0: 0.027380943298339844, 1: 0.01671886444091797, 2: 0.03119802474975586, 3: 0.01661205291748047, 4: 0.01943492889404297, 5: 0.01607513427734375, 6: 0.015127897262573242, 7: 0.016117095947265625, 8: 0.09782004356384277, 9: 0.017496109008789062, 10: 0.01684093475341797, 11: 0.021216869354248047, 12: 0.02081584930419922, 13: 0.01858210563659668, 14: 0.02910614013671875, 15: 0.017590999603271484, 16: 0.02479100227355957, 17: 0.01491689682006836, 18: 0.015413999557495117, 19: 0.015949010848999023, 20: 0.018386125564575195, 21: 0.025137901306152344, 22: 0.020014047622680664, 23: 0.013790130615234375, 24: 0.03187990188598633, 25: 0.01549220085144043, 26: 0.015345096588134766, 27: 0.013854026794433594, 28: 0.015204906463623047, 29: 0.014298200607299805, 30: 0.019289016723632812, 31: 0.016185998916625977, 32: 0.016319990158081055, 33: 0.014754056930541992, 34: 0.015031099319458008, 35: 0.014491081237792969, 36: 0.014528036117553711, 37: 0.015356063842773438, 38: 0.017026901245117188, 39: 0.015